# Random Search

This notebook contains the random search algorithm used in the paper, to compare with the RL algorithms.

## Imports and functions

In [ ]:
import numpy as np
import scipy as sci
import pandas as pd
import random
import itertools
from typing import Literal, Optional, Union

In [ ]:
def tuple_or_list_to_str(list_tuple_list: list[Union[tuple, list]]) -> list[str]:
    """Store the elements of a list with tuples/lists into a list with strings.

    Parameters
    ----------
    list_tuple_list : list[Union[tuple, list]]
        List of tuples/lists to store into a list of strs.

    Returns
    -------
    list[str]
        Elements of the tuple/list stored in a str.

    See Also
    --------
    str_to_tuple_or_list : function to undo this transformation.
    """
    return list(map(lambda x: " ".join(map(str, x)), list_tuple_list))

## Random Algorithm

In [ ]:
# Data imports 

vect = np.genfromtxt('../data/HTC/16layers_index.txt').astype(int) # Combination of materials
htc_vals = np.genfromtxt('../data/HTC/16layer_data.txt', dtype = 'float64') # Associated HTC

# Problem parmeters

N_trials = 1000 # Number of different random runs
N_search_steps = vect.shape[0] # Num ber of steps to perform per run (so far, until we find all states)

norm = 1e5 # Units for the HTC values

top5 = 1.56 # HTC between 5th and 6th best, used to determine if we have found a top 5 result
top1 = 1.61 # HTC between 1st and 2nd best, so we can determine if we have found the best

# Pre-allocations and definitions

indices = np.linspace(0,N_trials-1,N_trials)

htc_series = pd.Series(data = htc_vals, index = tuple_or_list_to_str(vect))

results_saved = [] # Will hold the pairs N_found_states-Best_state_found

x_elements = np.zeros((N_trials,N_search_steps)) # Will hold the number of found states
y_elements = np.zeros((N_trials,N_search_steps)) # Will hold the best state found
top5_vector = np.zeros((N_trials,N_search_steps)) # Will check if we have found top 5
top1_vector = np.zeros((N_trials,N_search_steps)) # Will check if we have found the best

# Random algorithm

for i in range(N_trials):
    
    seed = indices[i]
    
    random.seed(int(seed))
    
    random_all = list(itertools.product([0,1],repeat=16))
    
    random.shuffle(random_all)
    
    best_so_far = 0.0
    seen_so_far = []
    saved_so_far = []
    
    for j in range(N_search_steps):
                
        current_state_index = random_all[j]
        
        current_state_HTC = htc_series[tuple_or_list_to_str([current_state_index])][0]/(norm)
        
        if best_so_far > top5:
            
            top5_vector[i,j] = int(1)
            
        if best_so_far > top1:
            
            top1_vector[i,j] = int(i)
        
        if current_state_HTC not in seen_so_far:
            
            seen_so_far.append(current_state_HTC)
            
            if current_state_HTC > best_so_far:
                
                best_so_far = current_state_HTC
                            
        saved_so_far.append([len(seen_so_far),best_so_far])
        
    results_saved.append(saved_so_far)

# Computations

for i in range(N_trials):
    
    for j in range(N_search_steps):
        
        x_elements[i,j] = results_saved[i][j][0]
        y_elements[i,j] = results_saved[i][j][1]
    
x_to_plot = x_elements[0,:] # States found

y_mean = np.mean(y_elements,axis=0) # Mean of best state found across all runs
y_std = np.std(y_elements,axis=0)   # Std of best state found across all runs

top5_mean = np.mean(top5_vector,axis=0)*N_trials # Number of runs in which we found top 5 per N found states
top1_mean = np.mean(top1_vector,axis=0)*N_trials # Number of runs in which we found best per N found states